# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from types import SimpleNamespace

**Helper Function**

In [2]:
def event_space(event_list: list) -> SimpleNamespace:
    """Helper function which map
    list of csv columns names and values
    from line, than create SimpleNamespace
    which help to get values by dot
    ex. event.artist_name return name of artist from
    specific line.
    WARNING: Function ot change TYPES of csv values!
    Parameters:
    -----------
    event_list: list
        list of values to add into namespace, must be in same order as csv_names
    Returns:
    
        SimpleNamespace which make easier to get value for specific key
    -------
    
    """
    csv_names = ['artist_name', 'first_name', 'gender', 'itemin_session', 'last_name', 'lenght', 'level', 'location', 'session_id', 'song_title', 'user_id']
    event_dict = dict(zip(csv_names, event_list))
    event_namespace = SimpleNamespace(**event_dict)
    return event_namespace

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        # Change type of rows and add it to writer
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [6]:
# Create connection to local Apache Cassandra cluster
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    # make session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [7]:
# Creating KEYSPACE
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1}"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


### Create queries to ask the following three questions of the data

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [9]:
query = "CREATE TABLE IF NOT EXISTS song_library "
query = query + "(session_id int, itemin_session int, artist_name text, song_title text, song_lenght float, PRIMARY KEY (session_id, itemin_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [10]:
# Open event file and make query with appropriate row
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # create event space
        event = event_space(line)
        # make query
        query = "INSERT INTO song_library (session_id, itemin_session, artist_name, song_title, song_lenght)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        # execute query with appropriate columns
        session.execute(query, (int(event.session_id), int(event.itemin_session),
                                event.artist_name, event.song_title, float(event.lenght)))

#### Do a SELECT to verify that the data have been inserted into table

In [11]:
query = """
SELECT artist_name, song_title, song_lenght
FROM song_library 
WHERE session_id=338 AND itemin_session=4
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print ('artist: {}, song: {}, length: {}'.format(row.artist_name, row.song_title, row.song_lenght))

artist: Faithless, song: Music Matters (Mark Knight Dub), length: 495.30731201171875


#### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [12]:
# Create table for second query 
query = "CREATE TABLE IF NOT EXISTS users_library"
query = query + "(user_id int, session_id int, itemin_session int, first_name text, last_name text, artist_name text, song_title text, PRIMARY KEY((user_id, session_id), itemin_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [13]:
# Open event file and make query with appropriate row
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # create event space
        event = event_space(line)
        # make query
        query = "INSERT INTO users_library (user_id, session_id, itemin_session, first_name, last_name, artist_name, song_title)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        # execute query with appropriate columns
        session.execute(query, (int(event.user_id), int(event.session_id), int(event.itemin_session),
                                event.first_name, event.last_name, event.artist_name, event.song_title))

#### Do a SELECT to verify that the data have been inserted into table

In [14]:
query = """
SELECT artist_name, song_title, first_name, last_name
FROM users_library 
WHERE user_id=10 AND session_id=182
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print ('artist: {}, song: {}, full name: {} {}'.format(row.artist_name, row.song_title, row.first_name, row.last_name))

artist: Down To The Bone, song: Keep On Keepin' On, full name: Sylvie Cruz
artist: Three Drives, song: Greece 2000, full name: Sylvie Cruz
artist: Sebastien Tellier, song: Kilometer, full name: Sylvie Cruz
artist: Lonnie Gordon, song: Catch You Baby (Steve Pitron & Max Sanna Radio Edit), full name: Sylvie Cruz


#### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [15]:
# Create table for third query 
query = "CREATE TABLE IF NOT EXISTS song_listeners"
query = query + "(song_title text, user_id int, first_name text, last_name text, PRIMARY KEY (song_title, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [16]:
# Open event file and make query with appropriate row
file = 'event_datafile_new.csv'
with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # create event space
        event = event_space(line)
        # make query
        query = "INSERT INTO song_listeners (song_title, user_id, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        # execute query with appropriate columns
        session.execute(query, (event.song_title, int(event.user_id), event.first_name, event.last_name))

#### Do a SELECT to verify that the data have been inserted into table

In [17]:
query = """
SELECT first_name, last_name
FROM song_listeners 
WHERE song_title='All Hands Against His Own'
"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print (row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [21]:
for table_name in ['song_library', 'users_library', 'song_listeners']:
    query = 'DROP TABLE IF EXISTS {0}'.format(table_name)
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()